<font color="lightblue">

**GERMON Paul et MARTIN Hugo**

</font>

In [6]:
import Goban
import dataset_builder as db

In [7]:
import numpy as np
from sklearn.model_selection import train_test_split
import gzip, os.path
import json
%matplotlib inline
import matplotlib.pyplot as plt

In [8]:
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, BatchNormalization
import tensorflow.keras.optimizers as optimizers

In [9]:
data = db.get_raw_data_go("https://www.labri.fr/perso/lsimon/ia-inge2/samples-9x9.json.gz","samples-9x9.json.gz")
print("We have", len(data),"examples")

File samples-9x9.json.gz not found, I am downloading it... Done
We have 41563 examples


In [10]:
board_size = 9

X = db.build_dataset(data, board_size)
Y = db.get_winning_priors(data)

print(X.shape)
print(Y.shape)

(41563, 3, 9, 9)
(41563,)


In [11]:
enlarged_X, enlarged_Y = db.enlarge_dataset(X, Y)

print(enlarged_X.shape)
print(enlarged_Y.shape)

(332504, 3, 9, 9)
(332504,)


In [12]:
# Séparation des données d'entrée en un training set et un test set
X_train, X_test, Y_train, Y_test = train_test_split(enlarged_X, enlarged_Y, test_size=0.33, shuffle=True)

In [13]:
print(f"X.shape = {enlarged_X.shape}")
print(f"Y.shape = {enlarged_Y.shape}")
print(f"X_train.shape = {X_train.shape}")
print(f"X_test.shape = {X_test.shape}")
print(f"Y_train.shape = {Y_train.shape}")
print(f"Y_test.shape = {Y_test.shape}")

X.shape = (332504, 3, 9, 9)
Y.shape = (332504,)
X_train.shape = (222777, 3, 9, 9)
X_test.shape = (109727, 3, 9, 9)
Y_train.shape = (222777,)
Y_test.shape = (109727,)


# Model training

In [14]:
model_priors = Sequential()

# 1 Conv2D 128 filters
model_priors.add(Conv2D(filters = 128,
                 kernel_size = (3,3),
                 padding = 'same',
                 data_format = 'channels_first',
                 input_shape = X.shape[1:]))
model_priors.add(BatchNormalization())
model_priors.add(Activation('relu'))

# 3 Conv2D 64 filters
model_priors.add(Conv2D(filters = 64,
                 kernel_size = (3,3),
                 padding = 'same',
                 data_format = 'channels_first',
                 input_shape = X.shape[1:]))
model_priors.add(BatchNormalization())
model_priors.add(Activation('relu'))

model_priors.add(Conv2D(filters = 64,
                 kernel_size = (3,3),
                 padding = 'same',
                 data_format = 'channels_first',
                 input_shape = X.shape[1:]))
model_priors.add(BatchNormalization())
model_priors.add(Activation('relu'))

model_priors.add(Conv2D(filters = 64,
                 kernel_size = (3,3),
                 padding = 'same',
                 data_format = 'channels_first',
                 input_shape = X.shape[1:]))
model_priors.add(BatchNormalization())
model_priors.add(Activation('relu'))

# 5 Conv2D 32 filters
model_priors.add(Conv2D(filters = 32,
                 kernel_size = (3,3),
                 padding = 'same',
                 data_format = 'channels_first',
                 input_shape = X.shape[1:]))
model_priors.add(BatchNormalization())
model_priors.add(Activation('relu'))

model_priors.add(Conv2D(filters = 32,
                 kernel_size = (3,3),
                 padding = 'same',
                 data_format = 'channels_first',
                 input_shape = X.shape[1:]))
model_priors.add(BatchNormalization())
model_priors.add(Activation('relu'))

model_priors.add(Conv2D(filters = 32,
                 kernel_size = (3,3),
                 padding = 'same',
                 data_format = 'channels_first',
                 input_shape = X.shape[1:]))
model_priors.add(BatchNormalization())
model_priors.add(Activation('relu'))

model_priors.add(Conv2D(filters = 32,
                 kernel_size = (3,3),
                 padding = 'same',
                 data_format = 'channels_first',
                 input_shape = X.shape[1:]))
model_priors.add(BatchNormalization())
model_priors.add(Activation('relu'))

model_priors.add(Conv2D(filters = 32,
                 kernel_size = (3,3),
                 padding = 'same',
                 data_format = 'channels_first',
                 input_shape = X.shape[1:]))
model_priors.add(BatchNormalization())
model_priors.add(Activation('relu'))


# 3 Conv 2D 16 filters
model_priors.add(Conv2D(filters = 16,
                 kernel_size = (3,3),
                 padding = 'same',
                 data_format = 'channels_first',
                 input_shape = X.shape[1:]))
model_priors.add(BatchNormalization())
model_priors.add(Activation('relu'))

model_priors.add(Conv2D(filters = 16,
                 kernel_size = (3,3),
                 padding = 'same',
                 data_format = 'channels_first',
                 input_shape = X.shape[1:]))
model_priors.add(BatchNormalization())
model_priors.add(Activation('relu'))

model_priors.add(Conv2D(filters = 16,
                 kernel_size = (3,3),
                 padding = 'same',
                 data_format = 'channels_first',
                 input_shape = X.shape[1:]))
model_priors.add(BatchNormalization())
model_priors.add(Activation('relu'))

model_priors.add(Flatten())

# 3 Dense 160 neurons
model_priors.add(Dense(160))
model_priors.add(BatchNormalization())
model_priors.add(Activation('relu'))
model_priors.add(Dropout(0.5))

model_priors.add(Dense(160))
model_priors.add(BatchNormalization())
model_priors.add(Activation('relu'))
model_priors.add(Dropout(0.5))

model_priors.add(Dense(160))
model_priors.add(BatchNormalization())
model_priors.add(Activation('relu'))
model_priors.add(Dropout(0.5))

model_priors.add(Dense(1, activation='relu'))

model_priors.compile(loss='mse',
              optimizer='adam',
              metrics=['mae', 'mse'])

In [15]:
model_priors.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 9, 9)         3584      
_________________________________________________________________
batch_normalization (BatchNo (None, 128, 9, 9)         36        
_________________________________________________________________
activation (Activation)      (None, 128, 9, 9)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 9, 9)          73792     
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 9, 9)          36        
_________________________________________________________________
activation_1 (Activation)    (None, 64, 9, 9)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 9, 9)          3

In [16]:
epochs = 30
batch_size = 128

history = model_priors.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, Y_test),
          shuffle=True)

Epoch 1/30
1741/1741 [==============================] - 24s 14ms/step - loss: 0.1437 - mae: 0.2977 - mse: 0.1437 - val_loss: 0.0977 - val_mae: 0.2653 - val_mse: 0.0977
Epoch 2/30
1741/1741 [==============================] - 24s 14ms/step - loss: 0.0693 - mae: 0.1947 - mse: 0.0693 - val_loss: 0.0703 - val_mae: 0.1895 - val_mse: 0.0703
Epoch 3/30
1741/1741 [==============================] - 24s 14ms/step - loss: 0.0467 - mae: 0.1494 - mse: 0.0467 - val_loss: 0.1037 - val_mae: 0.2155 - val_mse: 0.1037
Epoch 4/30
1741/1741 [==============================] - 24s 14ms/step - loss: 0.0416 - mae: 0.1384 - mse: 0.0416 - val_loss: 0.1304 - val_mae: 0.2593 - val_mse: 0.1304
Epoch 5/30
1741/1741 [==============================] - 24s 14ms/step - loss: 0.0387 - mae: 0.1321 - mse: 0.0387 - val_loss: 0.0405 - val_mae: 0.1187 - val_mse: 0.0405
Epoch 6/30
1741/1741 [==============================] - 25s 14ms/step - loss: 0.0357 - mae: 0.1258 - mse: 0.0357 - val_loss: 0.0463 - val_mae: 0.1363 - val_mse:

In [17]:
score = model_priors.evaluate(X_test, Y_test, verbose=0)

In [18]:
print(len(score))
print('Test loss:', score[0])
print('Test mae:', score[1])
print('Test mse:', score[2])

3
Test loss: 0.03284333273768425
Test mae: 0.10987913608551025
Test mse: 0.03284333273768425


In [19]:
path = "C:/Users/Lenovo/Documents/Paul/Travail/ENSEIRB-MATMECA/3A/Alpha-Go/alpha-go-like/priors/"
model_priors.save('model_priors')

model_priors_json = model_priors.to_json()
with open("model_priors.json", "w") as json_file:
    json_file.write(model_priors_json)
# serialize weights to HDF5
model_priors.save_weights("model_priors.h5")
print("Saved model to disk")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: model_priors/assets
Saved model to disk


In [20]:
X_10 = X_test[:10]
Y_10 = Y_test[:10]
print(X_10.shape)
print(Y_10.shape)

Y_pred_10 = model_priors.predict(X_10)

for i in range(10):
  print(f"Sample {i}: predicted = {Y_pred_10[i]} / true = {Y_10[i]}")

(10, 3, 9, 9)
(10,)
Sample 0: predicted = [0.6540259] / true = 0.53
Sample 1: predicted = [0.] / true = 0.0
Sample 2: predicted = [0.] / true = 0.0
Sample 3: predicted = [0.41661206] / true = 0.23
Sample 4: predicted = [0.] / true = 0.01
Sample 5: predicted = [0.5693151] / true = 0.43
Sample 6: predicted = [0.5636389] / true = 0.46
Sample 7: predicted = [0.] / true = 0.0
Sample 8: predicted = [0.9462801] / true = 1.0
Sample 9: predicted = [0.9470581] / true = 1.0
